In [22]:
import json
import logging
import math

import numpy as np
from sentence_transformers import CrossEncoder, LoggingHandler, InputExample, losses
from sentence_transformers.evaluation import SentenceEvaluator, RerankingEvaluator
import torch
from torch.utils.data import DataLoader, Dataset
from tqdm.auto import tqdm


In [2]:
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])

logger = logging.getLogger("notebook")

In [3]:
with open("scripts/retrieval_results_wide_baseline_train.txt") as f:
    baseline_pids = [line.strip().split("####")[1].split("\t") for line in f.readlines()]

with open("data/mdd_all/dd-generation-structure/train.pids") as f:
    correct_pids = [line.strip() for line in f.readlines()]

with open("data/mdd_all/dd-generation-structure/train.source") as f:
    sources = [line.strip() for line in f.readlines()]

with open("data/mdd_dpr/dpr.psg.multidoc2dial_all.structure.json") as f:
    passages = json.load(f)

In [4]:
with open("scripts/retrieval_results_baseline.txt") as f:
    baseline_val_pids = [line.strip().split("####")[1].split("\t") for line in f.readlines()]

with open("data/mdd_all/dd-generation-structure/val.pids") as f:
    val_pids = [line.strip() for line in f.readlines()]

with open("data/mdd_all/dd-generation-structure/val.source") as f:
    val_sources = [line.strip() for line in f.readlines()]

In [5]:
passage_map = {p["id"]: p for p in passages}

In [6]:
def textify_ctx_instance(ctx, question):
    return question + " // " + ctx["title"] + " // " + ctx["text"]

In [7]:
train_examples = []
neg_pos_ratios = []
correct_rank = []

for i, (retrieved_pids, correct_pid, source) in enumerate(zip(baseline_pids, correct_pids, sources)):
    if correct_pid not in retrieved_pids:
        incorrect_pids = retrieved_pids
        correct_rank.append(None)
    elif (correct_idx := retrieved_pids.index(correct_pid)) != 0:
        incorrect_pids = retrieved_pids[:correct_idx]
    else:
        incorrect_pids = retrieved_pids[:5]

    neg_pos_ratios.append(len(incorrect_pids))

    for pid in incorrect_pids:
        instance = InputExample(texts=(source, textify_ctx_instance(passage_map[int(correct_pid)], source), textify_ctx_instance(passage_map[int(pid)], source)))
        train_examples.append(instance)


class TripletDataset(Dataset):
    def __init__(self, instances):
        self.instances = instances
    
    def __len__(self):
        return len(self.instances)

    def __getitem__(self, idx):
        return self.instances[idx]

In [9]:
DEFAULT_K_CUTOFFS = [1,10,30]

class RerankingPrecisionEvaluator(SentenceEvaluator):
    
    def __init__(self, samples, k_cutoffs = None, batch_size: int = 64, n_docs=30):
        self.samples = samples
        if k_cutoffs is None:
            self.k_cutoffs = DEFAULT_K_CUTOFFS
        else:
            self.k_cutoffs = k_cutoffs
        
        self.batch_size = batch_size

        if isinstance(self.samples, dict):
            self.samples = list(self.samples.values())

        ### Remove sample with empty positive / negative set
        self.samples = [sample for sample in self.samples if len(sample['positive']) > 0 and len(sample['negative']) > 0]
        self.failure_offset = len([sample for sample in self.samples if len(sample['positive']) == 0 or len(sample['negative']) == 0])
        self.masks = [torch.Tensor([True] * k + [False] * (n_docs - k)) for k in self.k_cutoffs]
        self.name="RerankingPrecisionEvaluator"

    def __call__(self, model, epoch: int = -1, steps: int = -1, **kwargs):
        logger.info("RerankingEvaluator: Evaluating the model on " + self.name)

        #### Some stats about the dataset
        num_positives = [len(sample['positive']) for sample in self.samples]
        num_negatives = [len(sample['negative']) for sample in self.samples]
        logger.info("Queries: {} \t Positives: Min {:.1f}, Mean {:.1f}, Max {:.1f} \t Negatives: Min {:.1f}, Mean {:.1f}, Max {:.1f}".format(len(self.samples), np.min(num_positives), np.mean(num_positives),
                                                                                                                                             np.max(num_positives), np.min(num_negatives),
                                                                                                                                             np.mean(num_negatives), np.max(num_negatives)))
        scores = self.compute_metrics(model)
        prec_at_1 = scores[1]
        prec_at_10 = scores[10]
        prec_at_30 = scores [30]

        logger.info(f"PID Prec@1: {prec_at_1:.2f}")
        logger.info(f"PID Prec@10: {prec_at_10:.2f}")
        logger.info(f"PID Prec@30: {prec_at_30:.2f}")
        
        return prec_at_10

    def compute_metrics(self, model):
        precisions = {k: 0 for k in self.k_cutoffs}

        for sample in tqdm(self.samples):
            n_docs = len(sample["positive"]) + len(sample["negative"])
            queries = [sample["query"] * n_docs]
            all_docs = sample["positive"] + sample["negative"]
            scores = model.predict([queries, all_docs])
            sorted_docs = torch.argsort(scores, descending=True)
            for k, mask in zip(self.k_cutoffs, self.masks):
                if bool(torch.any(sorted_docs[mask]) == 0):
                    precisions[k] += 1
            
        for k, precision_count in precisions.items():
            precisions[k] = precision_count / (len(self.samples) + self.failure_offset)

        return precisions
    

                

            


In [10]:
evaluation_samples = []
misses = []

for i, (retrieved_pids, val_pid, source) in enumerate(zip(baseline_val_pids, val_pids, val_sources)):
    incorrect_pids = [pid for pid in retrieved_pids if pid != val_pid]
    if val_pid not in retrieved_pids:
        misses.append(i)
        continue
    evaluation_samples.append({
        "query": source,
        "positive": [textify_ctx_instance(passage_map[int(val_pid)], source)],
        "negative": [textify_ctx_instance(passage_map[int(pid)], source) for pid in incorrect_pids]
    })


In [23]:
evaluator = RerankingPrecisionEvaluator(samples=evaluation_samples, k_cutoffs=[1,10,30], n_docs=30)
alt_evaluator = RerankingEvaluator(samples=evaluation_samples, )

In [12]:
len(train_examples), len(evaluation_samples)

(103397, 3024)

In [13]:
model = CrossEncoder("cross-encoder/ms-marco-TinyBERT-L-2-v2", device=4)

In [16]:
train_dataset = TripletDataset(train_examples)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=64)
train_loss = losses.MultipleNegativesRankingLoss(model=model)

In [24]:

model.fit(train_dataloader=train_dataloader,
          evaluator=alt_evaluator,
          evaluation_steps=20,
          epochs=4,
          warmup_steps= math.ceil(len(train_dataloader) * 4 * 0.1), #10% of train data for warm-up,
          use_amp=True,
          output_path="checkpoints/miniLM_triplet_finetuned",
          )

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1616 [00:00<?, ?it/s]

TypeError: Can't convert ['do not[SEP]agent: Do you want to know what is a standard driving record?||user: Hello I need information about How to get my own driving record // How to get my own driving record (abstract)#1_0 // How to get my own driving record (abstract) //   Related PDFs:', 'Sounds ok to me, I guess.[SEP]agent: You will receve a refund, *if* you cancel the appeal request before it is processed.||user: What if the appeal doesn\'t meet DMV requirements but I already paid?||agent: You can check the status of all appeals on our website. If it wasn\'t filed quickly enough, please contact a local branch.||user: How do I check the status of the appeal filed online?||agent: It depends on the alternate delivery method such as if you need the internet then you would need a computer to finish the course.||user: Is there any special equipment or technology I need to take an I-PIRP course?||agent: They will have to give you the option to receive a full refund of all course fees or re

In [ ]:
model([['query1', 'query2'], ['query1', 'query1']])